# Adversarial Autoencoder

ref. 

In [1]:
import numpy as np
import matplotlib.pyplot as plt

In [2]:
from keras.datasets import mnist
from keras.layers import Input, Dense, Reshape, Flatten
# from keras.layers import merge
from keras.layers import Lambda, Add
from keras.layers.advanced_activations import LeakyReLU
from keras.models import Sequential, Model
from keras.optimizers import Adam
import keras.backend as K

Using TensorFlow backend.


# Encoder

In [3]:
def build_encoder(img_shape, latent_dim):
    # Encoder

    img = Input(shape=img_shape)

    h = Flatten()(img)
    h = Dense(512)(h)
    h = LeakyReLU(alpha=0.2)(h)
    h = Dense(512)(h)
    h = LeakyReLU(alpha=0.2)(h)
    mu = Dense(latent_dim)(h)
    log_var = Dense(latent_dim)(h)
    
    # NOTE: original code uses keras.layer.merge() function which is deprecated
    #     latent_repr = merge([mu, log_var],
    #                          mode=lambda p: p[0] + K.random_normal(K.shape(p[0])) * K.exp(p[1] / 2),
    #                          output_shape=lambda p: p[0])
    
    # NOW: we use Lambda layer to do the same thing
    latent_repr = Lambda(lambda p: p[0] + K.random_normal(K.shape(p[0])) * K.exp(p[1] / 2), 
                         output_shape=lambda p: p[0])([mu, log_var])   
    
    return Model(img, latent_repr)

# Generator

In [4]:
def build_decoder(img_shape, latent_dim):

    model = Sequential()

    model.add(Dense(512, input_dim=latent_dim))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dense(512))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dense(np.prod(img_shape), activation='tanh'))
    model.add(Reshape(img_shape))

    model.summary()

    z = Input(shape=(latent_dim,))
    img = model(z)

    return Model(z, img)

# Discriminator

In [5]:
def build_discriminator(latent_dim):

    model = Sequential()

    model.add(Dense(512, input_dim=latent_dim))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dense(256))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dense(1, activation="sigmoid"))
    model.summary()

    encoded_repr = Input(shape=(latent_dim, ))
    validity = model(encoded_repr)

    return Model(encoded_repr, validity)

# Save images

In [6]:
def sample_images(epoch, decoder, latent_dim):
    r, c = 5, 5

    z = np.random.normal(size=(r*c, latent_dim))
    gen_imgs = decoder.predict(z)

    gen_imgs = 0.5 * gen_imgs + 0.5

    fig, axs = plt.subplots(r, c)
    cnt = 0
    for i in range(r):
        for j in range(c):
            axs[i,j].imshow(gen_imgs[cnt, :,:,0], cmap='gray')
            axs[i,j].axis('off')
            cnt += 1
    fig.savefig("images/mnist_%d.png" % epoch)
    plt.close()

# Save model

In [7]:
def save(model, model_name):
    model_path = "saved_model/%s.json" % model_name
    weights_path = "saved_model/%s_weights.hdf5" % model_name
    options = {"file_arch": model_path,
                "file_weight": weights_path}
    json_string = model.to_json()
    open(options['file_arch'], 'w').write(json_string)
    model.save_weights(options['file_weight'])

def save_model(G, D):
    save(G, "aae_generator")
    save(D, "aae_discriminator")

# Train the GAN

In [8]:
def train(encoder, decoder, adversarial_autoencoder, D, latent_dim, epochs, batch_size=128, sample_interval=50):

    # Load the dataset
    (X_train, _), (_, _) = mnist.load_data()

    # Rescale -1 to 1
    X_train = (X_train.astype(np.float32) - 127.5) / 127.5
    X_train = np.expand_dims(X_train, axis=3)

    # Adversarial ground truths
    valid = np.ones((batch_size, 1))
    fake = np.zeros((batch_size, 1))

    for epoch in range(epochs):

        # ---------------------
        #  Train Discriminator
        # ---------------------

        # Select a random batch of images
        idx = np.random.randint(0, X_train.shape[0], batch_size)
        imgs = X_train[idx]

        latent_fake = encoder.predict(imgs)
        latent_real = np.random.normal(size=(batch_size, latent_dim))

        # Train the discriminator
        d_loss_real = D.train_on_batch(latent_real, valid)
        d_loss_fake = D.train_on_batch(latent_fake, fake)
        d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

        # ---------------------
        #  Train Adversarial autoencoder
        # ---------------------

        # Train the combined stack
        g_loss = adversarial_autoencoder.train_on_batch(imgs, [imgs, valid])

        # Plot the progress
        print ("%d [D loss: %f, acc: %.2f%%] [G loss: %f, mse: %f]" % (epoch, d_loss[0], 100*d_loss[1], g_loss[0], g_loss[1]))

        # If at save interval => save generated image samples
        if epoch % sample_interval == 0:
            sample_images(epoch, decoder, latent_dim)

# main()

In [9]:
img_rows = 28
img_cols = 28
channels = 1
img_shape = (img_rows, img_cols, channels)
latent_dim = 10

In [10]:
optimizer = Adam(0.0002, 0.5)

Instructions for updating:
If using Keras pass *_constraint arguments to layers.


In [11]:
# Build and compile the discriminator
D = build_discriminator(latent_dim)
D.compile(loss='binary_crossentropy',
    optimizer=optimizer,
    metrics=['accuracy'])

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 512)               5632      
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 512)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 256)               131328    
_________________________________________________________________
leaky_re_lu_2 (LeakyReLU)    (None, 256)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 257       
Total params: 137,217
Trainable params: 137,217
Non-trainable params: 0
_________________________________________________________________
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [12]:
# Build the encoder / decoder
encoder = build_encoder(img_shape, latent_dim)
decoder = build_decoder(img_shape, latent_dim)

img = Input(shape=img_shape)

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_8 (Dense)              (None, 512)               5632      
_________________________________________________________________
leaky_re_lu_5 (LeakyReLU)    (None, 512)               0         
_________________________________________________________________
dense_9 (Dense)              (None, 512)               262656    
_________________________________________________________________
leaky_re_lu_6 (LeakyReLU)    (None, 512)               0         
_________________________________________________________________
dense_10 (Dense)             (None, 784)               402192    
_________________________________________________________________
reshape_1 (Reshape)          (None, 28, 28, 1)         0         
Total params: 670,480
Trainable params: 670,480
Non-trainable params: 0
________________________________________________

In [13]:
# The generator takes the image, encodes it and reconstructs it
# from the encoding
encoded_repr = encoder(img)
reconstructed_img = decoder(encoded_repr)

In [14]:
# For the adversarial_autoencoder model we will only train the generator
D.trainable = False

In [15]:
# The discriminator determines validity of the encoding
validity = D(encoded_repr)

In [16]:
# The adversarial_autoencoder model  (stacked generator and discriminator)
adversarial_autoencoder = Model(img, [reconstructed_img, validity])
adversarial_autoencoder.compile(loss=['mse', 'binary_crossentropy'],
    loss_weights=[0.999, 0.001],
    optimizer=optimizer)

In [17]:
epochs=20000
train(encoder, decoder, adversarial_autoencoder, D, 
      latent_dim, epochs=epochs, batch_size=32, sample_interval=200)

c:\users\henri\appdata\local\programs\python\python37\lib\site-packages\keras\engine\training.py:297: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


0 [D loss: 0.709242, acc: 45.31%] [G loss: 0.932656, mse: 0.932892]


c:\users\henri\appdata\local\programs\python\python37\lib\site-packages\keras\engine\training.py:297: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


1 [D loss: 0.723485, acc: 40.62%] [G loss: 0.911585, mse: 0.911848]
2 [D loss: 0.678178, acc: 65.62%] [G loss: 0.887497, mse: 0.887596]
3 [D loss: 0.594244, acc: 79.69%] [G loss: 0.830845, mse: 0.830549]
4 [D loss: 0.446154, acc: 84.38%] [G loss: 0.753564, mse: 0.752401]
5 [D loss: 0.339519, acc: 82.81%] [G loss: 0.632014, mse: 0.629320]
6 [D loss: 0.331175, acc: 79.69%] [G loss: 0.513258, mse: 0.509065]
7 [D loss: 0.270855, acc: 95.31%] [G loss: 0.402327, mse: 0.396740]
8 [D loss: 0.276211, acc: 89.06%] [G loss: 0.368877, mse: 0.362771]
9 [D loss: 0.273819, acc: 93.75%] [G loss: 0.295460, mse: 0.288617]
10 [D loss: 0.270123, acc: 90.62%] [G loss: 0.301874, mse: 0.295219]
11 [D loss: 0.224651, acc: 98.44%] [G loss: 0.273517, mse: 0.266922]
12 [D loss: 0.228217, acc: 96.88%] [G loss: 0.243804, mse: 0.237222]
13 [D loss: 0.218987, acc: 96.88%] [G loss: 0.282458, mse: 0.276712]
14 [D loss: 0.177371, acc: 100.00%] [G loss: 0.301844, mse: 0.296407]
15 [D loss: 0.204605, acc: 100.00%] [G los

120 [D loss: 0.321990, acc: 84.38%] [G loss: 0.171502, mse: 0.166168]
121 [D loss: 0.251563, acc: 85.94%] [G loss: 0.134479, mse: 0.128218]
122 [D loss: 0.210165, acc: 90.62%] [G loss: 0.167474, mse: 0.162526]
123 [D loss: 0.239953, acc: 92.19%] [G loss: 0.165101, mse: 0.161303]
124 [D loss: 0.302739, acc: 82.81%] [G loss: 0.159911, mse: 0.153943]
125 [D loss: 0.264143, acc: 87.50%] [G loss: 0.145432, mse: 0.139163]
126 [D loss: 0.272574, acc: 85.94%] [G loss: 0.152398, mse: 0.147149]
127 [D loss: 0.268790, acc: 87.50%] [G loss: 0.158343, mse: 0.152506]
128 [D loss: 0.337632, acc: 85.94%] [G loss: 0.154373, mse: 0.149531]
129 [D loss: 0.219743, acc: 87.50%] [G loss: 0.140158, mse: 0.134129]
130 [D loss: 0.225157, acc: 90.62%] [G loss: 0.140873, mse: 0.135627]
131 [D loss: 0.282925, acc: 87.50%] [G loss: 0.143437, mse: 0.139320]
132 [D loss: 0.363475, acc: 82.81%] [G loss: 0.140026, mse: 0.135030]
133 [D loss: 0.180740, acc: 92.19%] [G loss: 0.164101, mse: 0.158616]
134 [D loss: 0.26904

...

19522 [D loss: 0.386273, acc: 81.25%] [G loss: 0.059760, mse: 0.058133]
19523 [D loss: 0.404780, acc: 82.81%] [G loss: 0.057939, mse: 0.056232]
19524 [D loss: 0.266527, acc: 90.62%] [G loss: 0.048758, mse: 0.046658]
19525 [D loss: 0.334202, acc: 85.94%] [G loss: 0.054499, mse: 0.052305]
19526 [D loss: 0.581340, acc: 73.44%] [G loss: 0.067419, mse: 0.065744]
19527 [D loss: 0.411794, acc: 84.38%] [G loss: 0.055019, mse: 0.053035]
19528 [D loss: 0.256070, acc: 89.06%] [G loss: 0.050549, mse: 0.048368]
19529 [D loss: 0.361077, acc: 85.94%] [G loss: 0.059793, mse: 0.057996]
19530 [D loss: 0.340426, acc: 89.06%] [G loss: 0.056569, mse: 0.054652]
19531 [D loss: 0.495000, acc: 76.56%] [G loss: 0.056258, mse: 0.054370]
19532 [D loss: 0.423058, acc: 81.25%] [G loss: 0.056116, mse: 0.054412]
19533 [D loss: 0.367616, acc: 85.94%] [G loss: 0.056816, mse: 0.054619]
19534 [D loss: 0.480040, acc: 78.12%] [G loss: 0.066720, mse: 0.065130]
19535 [D loss: 0.436826, acc: 81.25%] [G loss: 0.056378, mse: 0.

19641 [D loss: 0.440622, acc: 78.12%] [G loss: 0.061288, mse: 0.059583]
19642 [D loss: 0.400811, acc: 84.38%] [G loss: 0.061764, mse: 0.059721]
19643 [D loss: 0.393420, acc: 84.38%] [G loss: 0.061878, mse: 0.060128]
19644 [D loss: 0.495333, acc: 75.00%] [G loss: 0.064221, mse: 0.062110]
19645 [D loss: 0.499006, acc: 73.44%] [G loss: 0.057151, mse: 0.055220]
19646 [D loss: 0.453923, acc: 81.25%] [G loss: 0.057848, mse: 0.055733]
19647 [D loss: 0.377395, acc: 78.12%] [G loss: 0.055216, mse: 0.053445]
19648 [D loss: 0.507369, acc: 78.12%] [G loss: 0.047867, mse: 0.045592]
19649 [D loss: 0.361232, acc: 82.81%] [G loss: 0.047387, mse: 0.045359]
19650 [D loss: 0.319665, acc: 85.94%] [G loss: 0.058568, mse: 0.056387]
19651 [D loss: 0.295677, acc: 89.06%] [G loss: 0.052839, mse: 0.051028]
19652 [D loss: 0.403870, acc: 76.56%] [G loss: 0.059966, mse: 0.058270]
19653 [D loss: 0.489147, acc: 78.12%] [G loss: 0.056622, mse: 0.054960]
19654 [D loss: 0.369133, acc: 84.38%] [G loss: 0.052604, mse: 0.

19761 [D loss: 0.379814, acc: 84.38%] [G loss: 0.063417, mse: 0.061589]
19762 [D loss: 0.497119, acc: 78.12%] [G loss: 0.056132, mse: 0.054520]
19763 [D loss: 0.493528, acc: 76.56%] [G loss: 0.056474, mse: 0.054869]
19764 [D loss: 0.411177, acc: 82.81%] [G loss: 0.050615, mse: 0.048761]
19765 [D loss: 0.327189, acc: 82.81%] [G loss: 0.052662, mse: 0.050762]
19766 [D loss: 0.345977, acc: 84.38%] [G loss: 0.058769, mse: 0.056610]
19767 [D loss: 0.450802, acc: 78.12%] [G loss: 0.056273, mse: 0.054582]
19768 [D loss: 0.430029, acc: 81.25%] [G loss: 0.052038, mse: 0.050398]
19769 [D loss: 0.317987, acc: 89.06%] [G loss: 0.057760, mse: 0.056004]
19770 [D loss: 0.464636, acc: 75.00%] [G loss: 0.061968, mse: 0.060288]
19771 [D loss: 0.436941, acc: 81.25%] [G loss: 0.058723, mse: 0.056720]
19772 [D loss: 0.544930, acc: 75.00%] [G loss: 0.055816, mse: 0.053507]
19773 [D loss: 0.601821, acc: 76.56%] [G loss: 0.054734, mse: 0.053196]
19774 [D loss: 0.312679, acc: 90.62%] [G loss: 0.052757, mse: 0.

19876 [D loss: 0.392659, acc: 87.50%] [G loss: 0.059284, mse: 0.057016]
19877 [D loss: 0.371764, acc: 84.38%] [G loss: 0.061905, mse: 0.059805]
19878 [D loss: 0.362221, acc: 82.81%] [G loss: 0.069147, mse: 0.067169]
19879 [D loss: 0.426515, acc: 81.25%] [G loss: 0.055009, mse: 0.052870]
19880 [D loss: 0.386139, acc: 81.25%] [G loss: 0.052729, mse: 0.050787]
19881 [D loss: 0.438210, acc: 79.69%] [G loss: 0.057276, mse: 0.055351]
19882 [D loss: 0.468075, acc: 76.56%] [G loss: 0.052699, mse: 0.050979]
19883 [D loss: 0.514638, acc: 75.00%] [G loss: 0.058023, mse: 0.056217]
19884 [D loss: 0.440147, acc: 85.94%] [G loss: 0.062471, mse: 0.060320]
19885 [D loss: 0.363831, acc: 84.38%] [G loss: 0.055922, mse: 0.054071]
19886 [D loss: 0.355867, acc: 82.81%] [G loss: 0.051348, mse: 0.049261]
19887 [D loss: 0.327298, acc: 85.94%] [G loss: 0.056042, mse: 0.053978]
19888 [D loss: 0.498546, acc: 71.88%] [G loss: 0.052738, mse: 0.051267]
19889 [D loss: 0.375510, acc: 81.25%] [G loss: 0.047799, mse: 0.

19995 [D loss: 0.391250, acc: 85.94%] [G loss: 0.051815, mse: 0.049918]
19996 [D loss: 0.414929, acc: 79.69%] [G loss: 0.056520, mse: 0.054676]
19997 [D loss: 0.516799, acc: 73.44%] [G loss: 0.061847, mse: 0.060459]
19998 [D loss: 0.467837, acc: 76.56%] [G loss: 0.068325, mse: 0.066344]
19999 [D loss: 0.496879, acc: 82.81%] [G loss: 0.048978, mse: 0.047255]
